# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
# Data Science
import pandas as pd
import numpy as np

# API requests
import requests
import json
from pprint import pprint

#Data Visualization
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key


In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Acapulco de Juárez,16.8634,-99.8901,80.42,78,0,6.91,MX,1733103013
1,1,Constanța,44.1833,28.6500,39.88,74,100,17.94,RO,1733102904
2,2,Klyuchi,52.2667,79.1667,16.11,93,100,7.07,RU,1733103016
3,3,Fortuna,40.5982,-124.1573,49.12,83,0,4.61,US,1733103018
4,4,Martapura,-3.4167,114.8500,78.89,89,75,5.75,ID,1733103019


In [30]:
city_data_df['Date'] = pd.to_datetime(city_data_df['Date'], unit='s')
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Acapulco de Juárez,16.8634,-99.8901,80.42,78,0,6.91,MX,2024-12-02 01:30:13
1,1,Constanța,44.1833,28.6500,39.88,74,100,17.94,RO,2024-12-02 01:28:24
2,2,Klyuchi,52.2667,79.1667,16.11,93,100,7.07,RU,2024-12-02 01:30:16
3,3,Fortuna,40.5982,-124.1573,49.12,83,0,4.61,US,2024-12-02 01:30:18
4,4,Martapura,-3.4167,114.8500,78.89,89,75,5.75,ID,2024-12-02 01:30:19
...,...,...,...,...,...,...,...,...,...,...
577,577,Pointe-Noire,-4.7761,11.8635,78.87,94,75,5.75,CG,2024-12-02 01:42:32
578,578,Mtambile,-5.3833,39.7000,78.24,78,81,2.21,TZ,2024-12-02 01:42:33
579,579,Aswān,24.0934,32.9070,58.30,41,0,4.61,EG,2024-12-02 01:42:34
580,580,El Abiodh Sidi Cheikh,32.8930,0.5484,55.26,54,99,3.58,DZ,2024-12-02 01:42:35


In [34]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     582 non-null    int64         
 1   City        582 non-null    object        
 2   Lat         582 non-null    float64       
 3   Lng         582 non-null    float64       
 4   Max Temp    582 non-null    float64       
 5   Humidity    582 non-null    int64         
 6   Cloudiness  582 non-null    int64         
 7   Wind Speed  582 non-null    float64       
 8   Country     579 non-null    object        
 9   Date        582 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 45.6+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 1000,
    frame_height = 500,
    color = "City",
    size="Humidity",
    hover_cols=["City", "Country", "Max Temp", "Humidity"], # Add 'city' to the tooltip
)

# Display the map plot
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
# Assuming Ideal weather conditions:

ideal_temp_min = 60  # Minimum temperature in Fahrenheit
ideal_temp_max = 80  # Maximum temperature in Fahrenheit
ideal_humidity_min = 40  # Minimum humidity in percentage
ideal_humidity_max = 60  # Maximum humidity in percentage
ideal_wind_max = 10  # Maximum wind speed in mph

# Drop any rows with null values
ideal_weather_df = city_data_df.dropna()

# Filter the DataFrame based on the ideal weather conditions

ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= ideal_temp_min) & 
    (city_data_df["Max Temp"] <= ideal_temp_max) &
    (city_data_df["Humidity"] >= ideal_humidity_min) & 
    (city_data_df["Humidity"] <= ideal_humidity_max) &
    (city_data_df["Wind Speed"] <= ideal_wind_max) 
]

# Display the filtered DataFrame with cities that meet the ideal weather conditions
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,Nouadhibou,20.9310,-17.0347,71.58,49,95,6.91,MR,2024-12-02 01:31:21
70,70,Torit,4.4133,32.5678,74.91,59,31,2.15,SS,2024-12-02 01:31:41
113,113,Holualoa,19.6228,-155.9522,79.86,54,0,8.05,US,2024-12-02 01:32:35
151,151,Pāthardi,19.1667,75.1833,70.88,52,100,6.49,IN,2024-12-02 01:33:22
191,191,Whangarei,-35.7317,174.3239,74.68,47,21,7.36,NZ,2024-12-02 01:34:12
257,257,Nichinan,31.6000,131.3667,64.06,53,0,2.21,JP,2024-12-02 01:35:36
285,285,Kalabo,-14.9700,22.6814,70.54,58,25,4.45,ZM,2024-12-02 01:36:12
314,314,Barro Alto,-11.7608,-41.9117,72.54,60,14,3.00,BR,2024-12-02 01:36:49
316,316,Naze,28.3667,129.4833,70.72,53,23,6.02,JP,2024-12-02 01:36:53
320,320,Abra Pampa,-22.7205,-65.6970,63.91,44,100,7.74,AR,2024-12-02 01:36:57


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
hotel_rows = []

for index, row in ideal_weather_df.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    humidity = row["Humidity"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        if (response.status_code == 200):
            data = response.json()
            
            # Print the results
            results = data.get("features", [])
            
            # Resiliency/Error Handling
            if len(results) > 0:
                place = results[0]
        
                # normalize data
                address = place.get("properties", {}).get("formatted")
                name = place.get("properties", {}).get("name")
    
            
                # return object
                hotel_row = {
                    "City": city,
                    "Country": country,
                    "Lat": latitude,
                    "Lng": longitude,
                    "Humidity" : humidity,
                    "Hotel Name": name,
    
                }
            else:
                hotel_row = {
                    "City": city,
                    "Country": country,
                    "Lat": latitude,
                    "Lng": longitude,
                    "Humidity": humidity,
                    "Hotel Name": None,
                }
               
        else:
            print(f"Error: {response.status_code}, {response.text}")
            hotel_row = {
                "City": city,
                "Country": country,
                "Lat": latitude,
                "Lng": longitude,
                "Humidity" : humidity,
                "Hotel Name": None,

            }
    except Exception as e:
        print(e)
        hotel_row = {
                "City": city,
                "Country": country,
                "Lat": latitude,
                "Lng": longitude,
                "Humidity" : humidity,
                "Hotel Name": None,

            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [46]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Nouadhibou,MR,20.9310,-17.0347,49,Hotel valencia
1,Torit,SS,4.4133,32.5678,59,Safari Link
2,Holualoa,US,19.6228,-155.9522,54,Kona Hotel
3,Pāthardi,IN,19.1667,75.1833,52,None
4,Whangarei,NZ,-35.7317,174.3239,47,Grand Hotel
5,Nichinan,JP,31.6000,131.3667,53,None
6,Kalabo,ZM,-14.9700,22.6814,58,Golden Lodge Luxury Acomodation
7,Barro Alto,BR,-11.7608,-41.9117,60,None
8,Naze,JP,28.3667,129.4833,53,ビジネス旅館畠山
9,Abra Pampa,AR,-22.7205,-65.6970,44,Residencia El Norte


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000  # 10km radius
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Limit to 1 result (nearest hotel)
    "apiKey": geoapify_key  # Corrected key parameter
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)

        # Convert the API response to JSON format
        name_address = response.json()


        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        if "features" in name_address and len(name_address["features"]) > 0:
            hotel = name_address["features"][0]  # Get the first hotel            
            # Try to get the hotel name
            hotel_name = hotel['properties'].get('name', 'No hotel name available')
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    except requests.exceptions.RequestException as e:
        print(f"Error with the API request for {row['City']}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated hotel_df with hotel names
hotel_df



Starting hotel search
Nouadhibou - nearest hotel: Noadhibou Guesthouse
Torit - nearest hotel: Safari Link
Holualoa - nearest hotel: Kona Hotel
Pāthardi - nearest hotel: No hotel found
Whangarei - nearest hotel: Grand Hotel
Nichinan - nearest hotel: No hotel name available
Kalabo - nearest hotel: Golden Lodge Luxury Acomodation
Barro Alto - nearest hotel: No hotel found
Naze - nearest hotel: ビジネス旅館畠山
Abra Pampa - nearest hotel: Residencia El Norte
Bulawayo - nearest hotel: Selboune Hotel
Mirpur Sakro - nearest hotel: No hotel found
Santa Rosalía - nearest hotel: Sol y Mar
Dwārka - nearest hotel: The Dwarika Hotel
Shingū - nearest hotel: ホテルニューパレス
Wenchang - nearest hotel: No hotel found
Sibut - nearest hotel: No hotel found
Nkowakowa - nearest hotel: Tzaneen Country Lodge
Howli - nearest hotel: Prashanti Tourist Lodge
Al Ghayzah - nearest hotel: فندق تاج العرب
Burao - nearest hotel: Liiban hotel burco ( ENG:Shibiin )
Alta Gracia - nearest hotel: The Ritz-Carlton
Meizhou - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Nouadhibou,MR,20.9310,-17.0347,49,Noadhibou Guesthouse
1,Torit,SS,4.4133,32.5678,59,Safari Link
2,Holualoa,US,19.6228,-155.9522,54,Kona Hotel
3,Pāthardi,IN,19.1667,75.1833,52,No hotel found
4,Whangarei,NZ,-35.7317,174.3239,47,Grand Hotel
5,Nichinan,JP,31.6000,131.3667,53,No hotel name available
6,Kalabo,ZM,-14.9700,22.6814,58,Golden Lodge Luxury Acomodation
7,Barro Alto,BR,-11.7608,-41.9117,60,No hotel found
8,Naze,JP,28.3667,129.4833,53,ビジネス旅館畠山
9,Abra Pampa,AR,-22.7205,-65.6970,44,Residencia El Norte


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [52]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size="Humidity",
    hover_cols=["City", "Country", "Max Temp", "Humidity", "Hotel Name", "Country"],
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name,Country)